In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.decomposition import PCA
import os

## Options

In [2]:
# == Options ==
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

## Plotting Functions


In [3]:
def plot_correlation_heatmap(data: pd.DataFrame, columns: list):
    """
    Plot correlation heatmap of specific columns
    """
    correlation_matrix = data[columns].corr()
    plt.figure(figsize=(13, 12))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidth=0.5)
    plt.title("Correlation Heatmap for selected attributes")
    plt.show()

def plot_distributions(data: pd.DataFrame):
    """
    Plot distributions of numeric data
    """
    sns.set_style("whitegrid", {"grid_linestyle": "--"})
    columns = data.columns
    num_plots = len(columns)
    num_rows = math.ceil(num_plots / 2)
    num_columns = 2

    fig, axes = plt.subplots(num_rows, num_columns, figsize=(12, 12))
    axes = axes.ravel()
    for i, column in enumerate(columns):
        sns.histplot(data[column], ax=axes[i], kde=True)
        axes[i].set_title(f"Distribution of {column}")
        axes[i].set_xlabel(column)
        axes[i].grid(axis="y", linestyle="--", alpha=0.6)
        plt.xlim(xmin=0)

    if num_plots % 2 != 0:
        axes.flat[-1].set_visible(False)
   
    plt.tight_layout()
    plt.show()

def plot_individual_distributions(data: pd.DataFrame):
    """
    Plot individual distributions of numeric data
    """
    sns.set_style("whitegrid", {"grid_linestyle": "--"})
    columns = data.columns

    for column in columns:
        plt.figure(figsize=(6, 4))
        sns.histplot(data[column], kde=True)
        # plt.title(f"Distribution of {column}")
        plt.xlabel(column)
        plt.grid(axis="y", linestyle="--", alpha=0.6)
        plt.xlim(xmin=0)
        plt.tight_layout()

        output_dir = "figures"
        output_filename = os.path.join(output_dir, f"{column}_distribution.png")
        plt.savefig(output_filename)
        plt.close()

def plot_violin(data: pd.DataFrame, column: str):
    """
    Plot violin 
    """    
    plt.figure(figsize=(6, 6))
    sns.violinplot(y=column, data=data, orient="v")
    # plt.title(f"Violin Plot of {column}")
    plt.show()

# - PCA Analysis

def plot_PCA_variance(numbers: list, ratios: list):
    """
    Plot variance ratio
    """
    plt.grid(True)
    plt.plot(numbers, ratios, marker="o")
    plt.xlabel("n_components")
    plt.ylabel("Explained Variance Ratio")
    # plt.title("n_components vs. Explained Variance Ratio")
    plt.ylim(ymin=0)
    plt.xlim(xmin=0)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()


def plot_PCA_directions(data_numeric, numbers):
    """
    Plot directions
    """
    # Assumption: data is already normalized 
    n_plots = len(numbers)
    n_cols = min(n_plots, 2)
    n_rows = (n_plots + n_cols - 1) // n_cols

    fig_width = 4 * n_cols 
    fig_height = 4 * n_rows
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))
    fig.subplots_adjust(hspace=0.7)

    if n_plots == 1:
        axes = np.array([axes])
    
    component_colors = plt.cm.viridis(np.linspace(0, 1, max(numbers) + 1))

    for i, number in enumerate(numbers):
        row = i // n_cols
        col = i % n_cols

        pca = PCA(n_components=number)
        pca.fit_transform(data_numeric)
        principal_direction = pca.components_

        if n_rows > 1:
            ax = axes[row, col]
        else:
            ax = axes[col]

        # ax.set_title(f"Principal Directions for {number} Components")
        for j, direction in enumerate(principal_direction):
            color = component_colors[j]
            ax.quiver(0, 0, direction[0], direction[1], angles="xy", scale_units="xy", scale=1.0, color=color, label=f"Component {j + 1}")
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")

        ax.grid(linestyle="--", linewidth=0.5, alpha=0.6, color="gray")

    common_legend = fig.legend(handles=[plt.Line2D([0], [0], color=component_colors[i], label=f"Component {i + 1}") for i in range(max(numbers) + 1)], title="Components", loc="upper right", bbox_to_anchor=(1.2, 1))
    for handle in common_legend.legendHandles:
        handle.set_visible(True)

    for i in range(n_plots, n_rows * n_cols):
        fig.delaxes(axes.flatten()[i])

    # plt.suptitle("PCA Components Directions", fontsize=16)
    plt.tight_layout()
    plt.show()

def plot_individual_violin_plots(data: pd.DataFrame):
    """
    Plot individual violin plots of numeric data
    """
    sns.set_style("whitegrid", {"grid_linestyle": "--"})
    columns = data.columns

    for column in columns:
        plt.figure(figsize=(6, 6))
        sns.violinplot(y=column, data=data, orient="v", color="skyblue")
        # plt.title(f"Violin Plot of {column}")
        plt.xlabel(column)
        plt.grid(axis="y", linestyle="--", alpha=0.6)
        plt.tight_layout()

        output_dir = "figures"
        output_filename = os.path.join(output_dir, f"{column}_violin.png")
        plt.savefig(output_filename)
        plt.close()

In [4]:
# == Reading the dataset ==
dataset_path = "./data/SAheart.csv"
data = pd.read_csv(dataset_path)

In [5]:
# == Rename selected columns (if needed) ==
columns = []
# Replace _ if existing
data = data.rename(columns={column: column.replace("_%", "") for column in columns})

In [6]:
# == Remove id(row.names) column ==
data.drop("id", inplace=True, axis=1)

In [7]:
# == Print none values ==
data.isna().sum()

sbp          0
tobacco      0
ldl          0
adiposity    0
famhist      0
typea        0
obesity      0
alcohol      0
age          0
chd          0
dtype: int64

In [8]:
# == Display the first rows ==
data.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [9]:
# == Select numeric columns ==
# (all columns are numeric except from famhist)
data_numeric = data.select_dtypes(exclude="object")
data_numeric.info(verbose=True, show_counts=True)
summary_statistics = data_numeric.describe().apply(lambda s: s.apply(lambda x: format(x, "g"))).transpose()
print("Statistics", summary_statistics)
latex_table = summary_statistics.to_latex()
print("Latex table", latex_table)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sbp        462 non-null    int64  
 1   tobacco    462 non-null    float64
 2   ldl        462 non-null    float64
 3   adiposity  462 non-null    float64
 4   typea      462 non-null    int64  
 5   obesity    462 non-null    float64
 6   alcohol    462 non-null    float64
 7   age        462 non-null    int64  
 8   chd        462 non-null    int64  
dtypes: float64(5), int64(4)
memory usage: 32.6 KB
Statistics           count     mean       std   min     25%     50%      75%     max
sbp         462  138.327   20.4963   101     124     134      148     218
tobacco     462  3.63565   4.59302     0  0.0525       2      5.5    31.2
ldl         462  4.74032   2.07091  0.98  3.2825    4.34     5.79   15.33
adiposity   462  25.4067    7.7807  6.74  19.775  26.115  31.2275   42.49
typea       462  53.1039

In [10]:
# == Plot distribution of numeric data ==
# Split for plotting purposes
# plot_distributions(data=data_numeric.iloc[:, <range here>])
# plot_distributions(data=data_numeric.iloc[:, [column_k, column_k_1 .....]])

In [11]:
# == Plot violin ==
plot_individual_violin_plots(data_numeric)

In [ ]:
# == Plot heatmap ==
plot_correlation_heatmap(data=data_numeric, columns=data_numeric.columns)

In [ ]:
# == Pair plots ==
# sns.pairplot(data=data, x_vars=["column_1"], y_vars=["column_2"], hue="mode", markers=["o", "s"])
# plt.grid(True, linestyle="--")
# plt.title("Set title")
# !! Ignore titles as when we move them to overleaf, we will write captions

# == Bar plots ==
# sns.set_style("whitegrid") 
# plt.figure(figsize=(10, 5))
# plt.subplot(121)
# sns.barplot(data=data, x="column_1", y="column_2", hue="mode")
# plt.title("Set title")
# plt.gca().yaxis.grid(True, linestyle='--', alpha=0.8)

# plt.subplot(122)
# sns.barplot(data=data, x="column_1", y="column_2", hue="key")
# plt.title("Set title")
# plt.legend(title="Set legend title", loc='center left', bbox_to_anchor=(1, 0.5))

# Adjust the layout
# plt.tight_layout()
# plt.gca().yaxis.grid(True, linestyle='--', alpha=0.8)

# Show the plot
# plt.show()

## PCA Analysis

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pca

In [ ]:
# PCA Analysis Guide (looks helpful)
# https://www.youtube.com/watch?v=oiusrJ0btwA
# https://github.com/dataprofessor/code/blob/master/python/PCA_analysis.ipynb

In [ ]:
# == Remove outliers before PCA Analysis ==
# It may not be necessary
for column_name in data_numeric.columns:
    z_scores = (data_numeric[column_name] - data_numeric[column_name].mean()) / data_numeric[column_name].std()
    data_numeric = data_numeric[abs(z_scores) < 3]  
# Remove data points with |Z-score| > 3

In [ ]:
data_numeric.head()

In [ ]:
# == Normalize before PCA ==

scaler = StandardScaler()
# Trying out a different scaler
# scaler = MinMaxScaler() 
data_normalized = scaler.fit_transform(data_numeric)

In [ ]:
# == Number of PCA's and variance captured ==
# The number of components is experimental at this stage
# Choosing the correct number of principal components is crucial
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9]
variance_ratios = []
# Trying out the different numbers
for number in numbers:
  pca_ = PCA(n_components=number)
  pca_.fit_transform(data_normalized)
  variance_ratios.append(np.sum(pca_.explained_variance_ratio_))
  print(f"Number of components\t{number}\tTotal variance\t{sum(pca_.explained_variance_ratio_)}")

plot_PCA_variance(numbers=numbers, ratios=variance_ratios)

In [ ]:
# == PCA variance captured ==
model = pca.pca()
out = model.fit_transform(data_normalized)
model.plot()


plt.grid(linestyle="--", alpha=0.6, color="gray") 
plt.ylim(ymin=0)
plt.show()

In [ ]:
# == PCA directions ==
plot_PCA_directions(data_numeric=data_normalized, numbers=numbers)

In [ ]:
# == PCA variance ratios ==
explained_variance_ratios = pca_.explained_variance_ratio_
most_informative_pca = explained_variance_ratios.argmax()

# Print explained variance ratios for all PCs
for i, explained_variance_ratio in enumerate(explained_variance_ratios):
    print(f"PC_{i + 1}: {explained_variance_ratio:.4f}")

print(f"The most informative PCA is PCA_{most_informative_pca + 1} with an explained variance ratio of {explained_variance_ratios[most_informative_pca]:.4f}")

In [ ]:
# == Interpret PCA_1 and direction ==
factor_loadings_pc1 = pca_.components_[0]
feature_names = data_numeric.columns
feature_loadings = dict(zip(feature_names, factor_loadings_pc1))
sorted_features = sorted(feature_loadings.items(), key=lambda x: abs(x[1]), reverse=True)

# Print the features with the highest absolute factor loadings for PC1
print("Features contributing most to PC1:")
for feature, loading in sorted_features:
    print(f"{feature}: {loading:.4f}")

In [ ]:
# == Try out 3, 4, 5 PCAs ==
number = 3
pca_ = PCA(n_components=number)
pca_.fit(data_normalized)
data_pca = pca_.transform(data_normalized)
data_pca = pd.DataFrame(data_pca, columns=["PCA_" + str(i) for i in range(number)])

In [ ]:
# == Pairplotting PCAs ==
# This is a useless plot for the time being
number = len(data_pca)
targets = list(range(number))
colors = plt.cm.viridis(np.linspace(0, 1, number))
data_pca["Target"] = targets

sns.set_style("whitegrid")
sns.pairplot(data_pca, hue="Target", palette="viridis")
plt.show()

In [ ]:
# == Correlation between PCAs ==
plt.figure(figsize=(12, 8))
factor_loadings = pca_.components_.T * np.sqrt(pca_.explained_variance_)
sns.heatmap(factor_loadings, cmap='coolwarm', annot=True, fmt='.2f')
plt.xlabel("Principal Components")
plt.ylabel("Variables")
plt.title("Factor Loadings Heatmap")
plt.show()

## Statistical Analysis

In [ ]:
from scipy.stats import kstest
from scipy.stats import lognorm
from scipy.stats import shapiro

In [ ]:
# == Statistics ==
alpha = 0.05

columns = []
ks_results = []
shapiro_results = []
lognorm_results = []

for column in data_numeric.columns:
    data_column = data_numeric[column]

    ks_statistic, ks_p_value = kstest(data_column, "norm")
    
    shapiro_statistic, shapiro_p_value = shapiro(data_column)
    
    lognorm_params = lognorm.fit(data_column)
    lognorm_statistic, lognorm_p_value = kstest(data_column, "lognorm", lognorm_params)
    
    columns.append(column)
    ks_results.append(ks_p_value > alpha)
    shapiro_results.append(shapiro_p_value > alpha)
    lognorm_results.append(lognorm_p_value > alpha)

results_df = pd.DataFrame({
    "Attribute": columns,
    "Kolmogorov-Smirnov Test (Normal)": ks_results,
    "Shapiro-Wilk Test (Normal)": shapiro_results,
    "Log-Normality Test": lognorm_results
})

results_df["Kolmogorov-Smirnov Test (Normal)"] = results_df["Kolmogorov-Smirnov Test (Normal)"].map({True: "Yes", False: "No"})
results_df["Shapiro-Wilk Test (Normal)"] = results_df["Shapiro-Wilk Test (Normal)"].map({True: "Yes", False: "No"})
results_df["Log-Normality Test"] = results_df["Log-Normality Test"].map({True: "Yes", False: "No"})

latex_table = results_df.to_latex(index=False)
print(latex_table)